In [1]:
from sklearn.datasets import make_regression
from sklearn.cross_validation import train_test_split
from sklearn.svm import LinearSVC, SVC
import numpy as np
from mosek.fusion import *

def primal_soft_margin_svm(m,n,X,y,C):

    W=[]
    B=[]

    with Model() as M:

        w  = M.variable( n, Domain.unbounded())
        b  = M.variable( 1, Domain.unbounded())
        t  = M.variable( 1, Domain.unbounded())
        xi = M.variable( m, Domain.greaterThan(0.))

        expr_tmp = Expr.mulElm(y, Expr.sub(Expr.mul(X,w), Variable.repeat(b,m) ))

        # Linear Kernel
        M.constraint( Expr.add(xi, expr_tmp), Domain.greaterThan(1.))

        M.constraint( Expr.vstack( 1., t, w) , Domain.inRotatedQCone())

        for c in C:
            M.objective(ObjectiveSense.Minimize, Expr.add( t, Expr.mul(c, Expr.sum(xi) ) ) )
            M.solve()

            try:
                W.append(w.level())
                B.append(b.level())
            except:
                W.append([])
                B.append([])

    return W,B


def get_function(W,B):
    def f(x):
        sum = 0.
        assert len(W) == len(x)
        for i in range(0,len(x)):
            sum += x[i]*W[i]
            sum += B
        return sum
    return f


def score(f, X_test, y_test):
    error = 0.
    for i in range(0, len(X_test)):
        prediction = f(X_test[i])
        if isinstance(prediction,np.ndarray):
            prediction = prediction[0]
        error += pow((prediction - y_test[i]),2)
    return error/len(X_test)

# Data fitting factor
C = []
for i in range(1,20):
    C.append( 1. / 20 * i )

# Create data
X, y, true_coefficient = make_regression(n_samples=80, n_features=5,
                                          n_informative=5, noise=0, coef=True,
                                          random_state=20140210)
# Make test and train set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=20140210)

m = len(X_train)
n = len(X_train[0])

# Run this implementation
W,B = primal_soft_margin_svm(m,n,X_train.copy(),y_train.copy(),C)
for i in range(0,len(W)):
    f = get_function(W[i],B[i])
    score_val = score(f, X_test.copy(), y_test.copy())
    print 'Custom: Overfitting Factor:',C[i],', mean square test error:', score_val


linear_svc = LinearSVC(loss="hinge").fit(X_train.copy(), y_train.copy())
print 'SCIKIT: mean square test error:', score( linear_svc.predict, X_test.copy(),
                                                y_test.copy())

svc = SVC(kernel="linear").fit(X_train.copy(), y_train.copy())
print 'SCIKIT: mean square test error:', score( svc.predict, X_test.copy(), y_test.copy())


Custom: Overfitting Factor: 0.05 , mean square test error: 26521.3980894
Custom: Overfitting Factor: 0.1 , mean square test error: 26432.07145
Custom: Overfitting Factor: 0.15 , mean square test error: 26404.1971831
Custom: Overfitting Factor: 0.2 , mean square test error: 26400.4693495
Custom: Overfitting Factor: 0.25 , mean square test error: 26370.0143602
Custom: Overfitting Factor: 0.3 , mean square test error: 26339.5846031
Custom: Overfitting Factor: 0.35 , mean square test error: 26332.4076602
Custom: Overfitting Factor: 0.4 , mean square test error: 26315.4156606
Custom: Overfitting Factor: 0.45 , mean square test error: 26298.4422598
Custom: Overfitting Factor: 0.5 , mean square test error: 26298.4422676
Custom: Overfitting Factor: 0.55 , mean square test error: 26298.442261
Custom: Overfitting Factor: 0.6 , mean square test error: 26298.4422592
Custom: Overfitting Factor: 0.65 , mean square test error: 26298.4422647
Custom: Overfitting Factor: 0.7 , mean square test error: 26

/Users/mich/.local/lib/python2.7/site-packages/mosek/fusion/__init__.py:35926: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if   ((mx == None) or (v == None)):
/Users/mich/.local/lib/python2.7/site-packages/mosek/fusion/__init__.py:37122: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  if   ((a == None) or (e == None)):
